# 메시지 병합 (Merge Messages)

이 노트북에서는 **연속된 같은 유형의 메시지를 하나로 병합**하는 방법을 배웁니다.

## 왜 메시지를 병합할까요?

```
┌────────────────────────────────────────────────────────────────────┐
│                    메시지 병합이 필요한 상황                        │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   문제 상황:                                                       │
│   [System] 당신은 친절한 어시스턴트입니다.                         │
│   [System] 항상 농담으로 대답하세요.      ← 연속된 System 메시지   │
│   [Human] 어떤 피자가 맛있나요?                                    │
│   [Human] 어떤 햄버거가 맛있나요?         ← 연속된 Human 메시지    │
│                                                                    │
│   일부 LLM은 연속된 같은 유형의 메시지를 처리 못함!                │
│                                                                    │
│   해결:                                                            │
│   [System] 당신은 친절한 어시스턴트입니다. 항상 농담으로 대답하세요.│
│   [Human] 어떤 피자가 맛있나요? 어떤 햄버거가 맛있나요?            │
│                                                                    │
│   → 같은 유형의 연속 메시지를 하나로 합침!                         │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
!pip install -q langchain langchain-ollama

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. 연속된 메시지 준비

In [ ]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)

# 연속된 같은 유형의 메시지가 있는 대화
messages = [
    SystemMessage(content='당신은 친절한 어시스턴트입니다.'),
    SystemMessage(content='항상 농담으로 대답하세요.'),  # 연속된 System
    HumanMessage(
        content=[{'type': 'text', 'text': '어떤 피자가 제일 맛있나요?'}]
    ),
    HumanMessage(content='어떤 햄버거가 가장 맛있나요?'),  # 연속된 Human
    AIMessage(content='나는 항상 너만 "고르곤졸라"'),
    AIMessage(content='너가 "버거" 싶어'),  # 연속된 AI
]

print(f"=== 병합 전 ({len(messages)}개 메시지) ===")
for i, msg in enumerate(messages):
    content = str(msg.content)[:40]
    print(f"[{i+1}] {msg.type}: {content}..." if len(str(msg.content)) > 40 else f"[{i+1}] {msg.type}: {content}")

# 3. merge_message_runs 사용하기

In [ ]:
from langchain_core.messages import merge_message_runs

# 연속된 메시지 병합
merged = merge_message_runs(messages)

print(f"=== 병합 후 ({len(merged)}개 메시지) ===")
for i, msg in enumerate(merged):
    print(f"\n[{i+1}] {msg.type}:")
    print(f"    {msg.content}")

# 4. 병합 결과 자세히 보기

In [ ]:
print("=== 병합 전후 비교 ===")
print("\n[병합 전]")
print(f"  System 메시지: {len([m for m in messages if m.type == 'system'])}개")
print(f"  Human 메시지: {len([m for m in messages if m.type == 'human'])}개")
print(f"  AI 메시지: {len([m for m in messages if m.type == 'ai'])}개")

print("\n[병합 후]")
print(f"  System 메시지: {len([m for m in merged if m.type == 'system'])}개")
print(f"  Human 메시지: {len([m for m in merged if m.type == 'human'])}개")
print(f"  AI 메시지: {len([m for m in merged if m.type == 'ai'])}개")

# 5. 체인에서 merger 사용하기

In [ ]:
from langchain_ollama import ChatOllama

# LLM 모델
model = ChatOllama(model='llama3.2')

# merger 생성 (체인에서 사용할 때)
merger = merge_message_runs()

# 체인: 병합 → 모델
chain = merger | model

print("✅ merger가 포함된 체인 생성 완료")

In [ ]:
# 체인 실행
result = chain.invoke(messages)

print("=== 병합 체인 실행 결과 ===")
print(f"입력: {len(messages)}개 메시지")
print(f"병합 후: {len(merged)}개 메시지")
print(f"\nAI 응답: {result.content}")

# 6. 다양한 상황 테스트

In [ ]:
# 연속되지 않은 메시지는 병합되지 않음
alternating_messages = [
    HumanMessage(content='안녕하세요'),
    AIMessage(content='반가워요'),
    HumanMessage(content='오늘 날씨 어때요?'),
    AIMessage(content='좋아요!'),
]

merged_alternating = merge_message_runs(alternating_messages)

print("=== 번갈아 나오는 메시지 (병합 안됨) ===")
print(f"병합 전: {len(alternating_messages)}개")
print(f"병합 후: {len(merged_alternating)}개")
print("\n→ 연속되지 않아서 병합 안됨!")

In [ ]:
# 여러 개의 연속 메시지 테스트
many_consecutive = [
    HumanMessage(content='첫 번째 질문'),
    HumanMessage(content='두 번째 질문'),
    HumanMessage(content='세 번째 질문'),
    HumanMessage(content='네 번째 질문'),
]

merged_many = merge_message_runs(many_consecutive)

print("=== 4개 연속 Human 메시지 ===")
print(f"병합 전: {len(many_consecutive)}개")
print(f"병합 후: {len(merged_many)}개")
print(f"\n병합된 내용:\n{merged_many[0].content}")

---

## 정리: 메시지 병합 (merge_message_runs)

### 핵심 동작

```
┌────────────────────────────────────────────────────────────────────┐
│                merge_message_runs 동작 원리                        │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   [System] A                                                       │
│   [System] B    →  [System] A + B                                 │
│                                                                    │
│   [Human] 질문1                                                    │
│   [Human] 질문2  →  [Human] 질문1 + 질문2                          │
│                                                                    │
│   [AI] 답변1                                                       │
│   [AI] 답변2     →  [AI] 답변1 + 답변2                             │
│                                                                    │
│   → 같은 유형이 연속되면 내용을 합침                               │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

### 핵심 코드

```python
from langchain_core.messages import merge_message_runs

# 1. 직접 사용
merged = merge_message_runs(messages)

# 2. 체인에서 사용
merger = merge_message_runs()
chain = merger | model
result = chain.invoke(messages)
```

### 언제 사용할까요?

| 상황 | 사용 여부 |
|------|----------|
| 연속된 시스템 지시 통합 | ✅ 사용 |
| 여러 사용자 입력 통합 | ✅ 사용 |
| API 제한 우회 | ✅ 사용 |
| 번갈아 나오는 메시지 | ❌ 불필요 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
model = ChatOpenAI(model='gpt-4o-mini')

# 변경
model = ChatOllama(model='llama3.2')
```

## ch04 요약: 메모리 관리 기법

| 기법 | 용도 | 핵심 함수/클래스 |
|------|------|------------------|
| **Simple Memory** | 대화 기록 유지 | `placeholder` |
| **StateGraph** | 상태 기반 챗봇 | `StateGraph`, `add_messages` |
| **Persistent Memory** | 대화 저장/복원 | `MemorySaver`, Thread ID |
| **Trim Messages** | 토큰 제한 대응 | `trim_messages` |
| **Filter Messages** | 조건별 필터링 | `filter_messages` |
| **Merge Messages** | 연속 메시지 병합 | `merge_message_runs` |